<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-26 10:22:42
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: 1.16 L (0.82%)
Current PnL: -22.45 L (-14.71%)
CY Booked + Current PnL: -8.56 L (-5.61%)
-------------------
Total profit:  1.49 L
Total loss:  -23.93 L
-------------------
Total Booked + Current PnL: 18.42 L (14.67%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.69%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.86 L (61.29%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.5%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.70,-7.78,8.45,0.02,13603.0,-13576.0,160983.0,145.54,72.0,M-SC,7.88,234.0,-1.00,1.12,40.03,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.05,-14.06,16.46,0.09,14256.0,-14166.0,86611.0,101.07,50.0,M-SC,3.70,253.0,-0.99,0.60,13.79,OX40N,NTT,DURABLES
35,ICICIGI,2252.93,0.88,7.59,11.72,20.19,21980.0,13224.0,187544.0,-15.78,53.0,X-MC,7.48,68.0,0.60,1.31,23.85,X40,ATH,INSURANCE
53,NESTLEIND,1377.00,1.05,12.53,7.88,21.40,23536.0,33252.0,298678.0,4.42,58.0,X-LC,7.79,12.0,1.41,2.08,20.00,XY25,NTT,FMCG
83,VOLTAS,1530.00,1.22,7.39,11.45,19.69,23578.0,14178.0,205920.0,-3.15,50.0,X-MC,3.38,78.0,0.60,1.44,14.69,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TANLA,1943.92,3.06,-35.02,238.07,119.69,418884.0,-94811.0,175950.0,-34.29,36.0,H-SC,11.18,156.0,-0.23,1.23,39.00,AR,ATH,IT
66,SIEMENS,4671.50,2.95,-12.01,42.64,25.51,69823.0,-22345.0,163750.0,0.51,63.0,H-LC,1.86,49.0,-0.32,1.14,20.67,AR,ATH,ELECTRICAL
28,GREENPANEL,537.00,2.86,-30.59,106.22,43.14,134979.0,-56003.0,127075.0,184.88,37.0,M-SC,5.44,240.0,-0.41,0.89,19.53,XY24,NTT,MISC
0,5PAISA,593.00,2.73,-34.76,73.06,12.90,104141.0,-75954.0,142542.0,110.34,59.0,H-SC,13.99,161.0,-0.73,0.99,18.52,OX40N,NTT,FINANCE
64,SFL,1287.00,2.59,-40.13,111.52,26.63,175064.0,-105238.0,156980.0,18.85,34.0,M-SC,4.99,239.0,-0.60,1.10,2.59,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.84,-27.73,97.64,42.84,45873.0,-18024.0,46982.0,-52.62,41.0,X-SC,37.47,83.0,-0.39,0.33,2.23,XY24,NTT,MISC
44,JCHAC,2282.00,-1.82,-24.88,33.10,-0.01,31780.0,-31793.0,96012.0,19552.17,31.0,M-SC,1.60,233.0,-1.00,0.67,8.72,OX40N,NTT,AC
38,INDIAMART,4810.62,-1.76,-0.39,107.53,106.72,132105.0,-482.0,122854.0,-50.63,37.0,H-SC,3.36,139.0,-0.00,0.86,22.90,AR,ATH,MISC
12,BERGEPAINT,680.00,-1.35,0.02,21.11,21.14,48001.0,48.0,227387.0,-7.04,51.0,X-MC,3.98,74.0,0.00,1.59,28.17,XY24,NTT,PAINTS
62,ROUTE,2227.21,-1.03,-49.81,236.82,69.05,158163.0,-66280.0,66786.0,-59.29,36.0,H-SC,23.69,140.0,-0.42,0.47,3.00,SR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-1.76,-0.39,107.53,106.72,132105.0,-482.0,122854.0,-50.63,37.0,H-SC,3.36,139.0,-0.00,0.86,22.9,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,1.20,-0.92,36.88,35.63,76582.0,-1922.0,207652.0,-14.97,42.0,H-MC,4.17,119.0,-0.03,1.45,15.6,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.50,-4.10,116.79,107.91,165370.0,-6048.0,141596.0,-21.77,38.0,M-SC,10.46,216.0,-0.04,0.99,1.25,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.70,-7.78,8.45,0.02,13603.0,-13576.0,160983.0,145.54,72.0,M-SC,7.88,234.0,-1.00,1.12,40.03,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.05,-14.06,16.46,0.09,14256.0,-14166.0,86611.0,101.07,50.0,M-SC,3.70,253.0,-0.99,0.60,13.79,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.12,-22.99,47.35,13.47,98334.0,-61992.0,207675.0,-68.31,25.0,H-SC,1.29,158.0,-0.63,1.45,5.60,XY24,NTT,PAINTS
67,SIS,528.00,0.81,-23.25,57.99,21.26,49420.0,-25811.0,85221.0,2011.92,51.0,H-SC,3.87,166.0,-0.52,0.59,15.14,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,0.97,1.73,69.34,72.27,119524.0,2926.0,172374.0,-12.89,57.0,M-LC,4.44,99.0,0.02,1.2,8.01,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.97,1.73,69.34,72.27,119524.0,2926.0,172374.0,-12.89,57.0,M-LC,4.44,99.0,0.02,1.20,8.01,XR,NTT,IT
38,INDIAMART,4810.62,-1.76,-0.39,107.53,106.72,132105.0,-482.0,122854.0,-50.63,37.0,H-SC,3.36,139.0,-0.00,0.86,22.90,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,1.20,-0.92,36.88,35.63,76582.0,-1922.0,207652.0,-14.97,42.0,H-MC,4.17,119.0,-0.03,1.45,15.60,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,0.51,-3.72,88.95,81.92,98316.0,-4269.0,110529.0,-43.65,37.0,M-SC,3.81,236.0,-0.04,0.77,30.85,XR,NTT,DURABLES
25,FINCABLES,1641.55,0.50,-4.10,116.79,107.91,165370.0,-6048.0,141596.0,-21.77,38.0,M-SC,10.46,216.0,-0.04,0.99,1.25,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.42,-35.36,109.16,35.20,90793.0,-45496.0,83174.0,9.03,24.0,X-SC,13.03,92.0,-0.50,0.58,0.17,X40,NTT,FOOTWEAR
46,JSWINFRA,342.00,0.99,-9.05,26.69,15.23,48633.0,-18131.0,182216.0,-22.11,29.0,X-MC,6.78,66.0,-0.37,1.27,20.28,X40N,NTT,REALTY
76,TMPV,600.00,2.11,-14.72,66.71,42.18,157258.0,-40676.0,235734.0,-23.08,29.0,X-LC,1.05,3.0,-0.26,1.64,2.11,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.23,-13.95,72.53,48.45,125135.0,-27978.0,172528.0,-20.09,30.0,X-MC,7.08,64.0,-0.22,1.20,0.23,X40N,ATH,FINANCE
55,PGHH,17907.65,-0.34,-4.54,40.12,33.76,76910.0,-9120.0,191700.0,-32.64,33.0,X-MC,3.55,57.0,-0.12,1.34,0.42,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.49,-1.60,12.23,10.44,24086.0,-3193.0,196945.0,-41.15,43.0,X-LC,0.89,5.0,-0.13,1.37,3.88,X40,NTT,FMCG
76,TMPV,600.0,2.11,-14.72,66.71,42.18,157258.0,-40676.0,235734.0,-23.08,29.0,X-LC,1.05,3.0,-0.26,1.64,2.11,XY24,NTT,AUTO
3,ACC,3906.0,0.22,-21.28,108.52,64.15,203280.0,-50631.0,187320.0,-54.13,54.0,X-SC,1.60,82.0,-0.25,1.31,5.08,XY24,BTT,CEMENT
36,ICICIPRULI,790.0,1.35,2.87,27.32,30.97,49669.0,5068.0,181806.0,-21.38,56.0,X-MC,1.69,75.0,0.10,1.27,15.79,X40,ATH,INSURANCE
66,SIEMENS,4671.5,2.95,-12.01,42.64,25.51,69823.0,-22345.0,163750.0,0.51,63.0,H-LC,1.86,49.0,-0.32,1.14,20.67,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.42,-35.36,109.16,35.20,90793.0,-45496.0,83174.0,9.03,24.0,X-SC,13.03,92.0,-0.50,0.58,0.17,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,0.23,-13.95,72.53,48.45,125135.0,-27978.0,172528.0,-20.09,30.0,X-MC,7.08,64.0,-0.22,1.20,0.23,X40N,ATH,FINANCE
55,PGHH,17907.65,-0.34,-4.54,40.12,33.76,76910.0,-9120.0,191700.0,-32.64,33.0,X-MC,3.55,57.0,-0.12,1.34,0.42,X40,ATH,FMCG
54,PAGEIND,51605.07,0.91,-5.01,32.71,26.07,50877.0,-8200.0,155540.0,-30.06,38.0,X-MC,9.19,55.0,-0.16,1.09,0.91,X40,ATH,APPARELS
76,TMPV,600.00,2.11,-14.72,66.71,42.18,157258.0,-40676.0,235734.0,-23.08,29.0,X-LC,1.05,3.0,-0.26,1.64,2.11,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.84,-27.73,97.64,42.84,45873.0,-18024.0,46982.0,-52.62,41.0,X-SC,37.47,83.0,-0.39,0.33,2.23,XY24,NTT,MISC
60,RELAXO,1176.00,1.71,-47.29,188.94,52.31,144569.0,-68644.0,76516.0,-43.86,34.0,X-SC,4.14,91.0,-0.47,0.53,2.42,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.42,-35.36,109.16,35.20,90793.0,-45496.0,83174.0,9.03,24.0,X-SC,13.03,92.0,-0.50,0.58,0.17,X40,NTT,FOOTWEAR
22,DIXON,18931.72,2.00,-5.66,28.88,21.58,38180.0,-7938.0,132201.0,-52.41,35.0,X-MC,6.73,56.0,-0.21,0.92,17.04,X40N,ATH,IT
52,MEDANTA,1486.00,0.99,5.37,17.75,24.08,23745.0,6822.0,133772.0,-3.08,50.0,X-SC,5.18,89.0,0.29,0.93,26.04,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.76,-14.60,39.68,19.28,117228.0,-50523.0,295433.0,-25.57,60.0,X-LC,7.36,1.0,-0.43,2.06,9.22,X40,ATH,IT
41,INFY,2275.00,0.84,5.07,47.39,54.87,155802.0,15878.0,328766.0,-18.16,58.0,X-LC,2.87,2.0,0.10,2.29,13.78,X40,BTT,IT
76,TMPV,600.00,2.11,-14.72,66.71,42.18,157258.0,-40676.0,235734.0,-23.08,29.0,X-LC,1.05,3.0,-0.26,1.64,2.11,XY24,NTT,AUTO
82,VBL,671.64,1.60,-7.84,47.21,35.67,137423.0,-24754.0,291088.0,-18.74,48.0,X-LC,6.01,4.0,-0.18,2.03,4.93,X40N,ATH,FMCG
43,ITC,452.00,0.49,-1.60,12.23,10.44,24086.0,-3193.0,196945.0,-41.15,43.0,X-LC,0.89,5.0,-0.13,1.37,3.88,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,2.08,-36.70,109.95,32.89,53731.0,-28338.0,48869.0,-703.29,65.0,L-MC,5.61,259.0,-0.53,0.34,34.58,XR,NTT,BANKS
6,ASIANTILES,137.00,0.74,-12.07,106.61,81.67,87943.0,-11320.0,82490.0,7416.67,54.0,L-SC,19.06,271.0,-0.13,0.58,59.74,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.61,-12.75,59.93,39.54,52895.0,-12898.0,88261.0,-31.70,65.0,M-SC,6.63,220.0,-0.24,0.62,24.46,AR,ATH,AUTO
51,MASFIN,398.61,1.01,-4.85,28.27,22.05,26355.0,-4755.0,93225.0,-17.80,51.0,H-SC,7.43,164.0,-0.18,0.65,35.51,XR,ATH,FINANCE
70,SURYODAY,216.00,0.64,-19.05,51.98,23.03,75351.0,-34109.0,144962.0,57.89,50.0,H-SC,10.13,167.0,-0.45,1.01,43.83,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.70,-7.78,8.45,0.02,13603.0,-13576.0,160983.0,145.54,72.0,M-SC,7.88,234.0,-1.00,1.12,40.03,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.49,1.01,19.73,20.95,50177.0,2552.0,254320.0,-0.07,77.0,X-LC,13.97,31.0,0.05,1.77,37.01,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,2.08,-36.70,109.95,32.89,53731.0,-28338.0,48869.0,-703.29,65.0,L-MC,5.61,259.0,-0.53,0.34,34.58,XR,NTT,BANKS
0,5PAISA,593.00,2.73,-34.76,73.06,12.90,104141.0,-75954.0,142542.0,110.34,59.0,H-SC,13.99,161.0,-0.73,0.99,18.52,OX40N,NTT,FINANCE
79,UNITDSPR,1644.00,0.41,5.50,14.47,20.77,35745.0,12882.0,247026.0,-3.68,60.0,X-MC,3.69,62.0,0.36,1.72,12.80,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.73
1,25,44.52
2,50,75.45


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.71
MC    29.20
LC    25.09
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.90
X40      21.16
X40N     11.87
XR        9.65
XY25      9.08
AR        9.05
OX40N     7.68
X200      1.84
SR        0.99
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.98
X-MC    22.76
X-LC    19.94
M-SC    12.07
X-SC     8.24
H-MC     4.72
H-LC     2.98
L-SC     1.42
M-MC     1.38
M-LC     1.20
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.74,-4.80,38.69
IT,13.72,-14.93,74.26
FINANCE,9.45,-14.41,62.76
MISC,7.46,-21.68,71.53
ELECTRICAL,5.95,-9.99,50.28
PAINTS,5.93,-8.58,24.71
INSURANCE,4.45,0.87,34.07
PHARMA,3.88,-0.49,32.65
AUTO,3.34,-20.17,71.70


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3039318.0,21
XR,1279376.0,13
AR,1276045.0,10
X40,1005403.0,14
X40N,744791.0,9
OX40N,688106.0,10
XY25,339928.0,6
SR,279718.0,2
MH,72337.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3542005.0,25
M-SC,1329659.0,15
X-MC,1220372.0,16
X-LC,902407.0,11
X-SC,738695.0,8
H-MC,401768.0,3
L-SC,258834.0,3
H-LC,130359.0,2
M-LC,119524.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1212376.0      6
           AR         872294.0      5
           XR         786802.0      7
M-SC       XY24       776556.0      6
X-MC       X40        436452.0      7
X-LC       X40        394339.0      5
X-MC       XY24       358230.0      3
H-SC       OX40N      318478.0      4
X-SC       X40N       291185.0      3
M-SC       OX40N      282468.0      5
H-SC       SR         279718.0      2
X-SC       XY24       272898.0      3
X-MC       X40N       247693.0      4
X-LC       XY24       228428.0      2
H-MC       AR         210938.0      2
X-LC       X40N       205913.0      2
H-MC       XY24       190830.0      1
X-MC       XY25       177997.0      2
X-SC       X40        174612.0      2
L-SC       XR         171674.0      2
M-SC       XR         147645.0      2
           AR         122990.0      2
M-LC       XR         119524.0      1
L-SC       OX40N       87160.0      1
X-LC       XY25        73727.0      2
H-SC       MH          72337.0      1
H-LC       AR          69823.0      1
           X200        60536.0      1
L-MC       XR          53731.0      1
M-MC       XY25        47684.0      1
L-LC       XY25        40520.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
